In [5]:
from imports import *
from utils import *
from model import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data loading

dataset = 'ECG'

# 使用函数创建数据集
data_dir = 'raw\ECG'
train_dataset, test_dataset = load_ecg_data(data_dir)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [6]:

# Define the number of classes for the classification task
num_classes = 2

# Initialize the model with the given configuration and number of classes, and move it to the specified device
model = NET_32k_ECG(num_classes).to(device)

total_weights = count_weights(model)
print(f"The network has {total_weights} weights that need to be deployed.")

model.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')
model = torch.quantization.prepare_qat(model, inplace=True)
model = torch.quantization.convert(model, inplace=True)

The network has 3647 weights that need to be deployed.


In [7]:
# Load Pretrained Model

# Specify the filename of the saved model
model_filename = f"checkpoint/ECG-3647-99.40_model.t7"

# Load the checkpoint from the file, mapping the model to the specified device
checkpoint = torch.load(model_filename, map_location=device)

# Load the model state dictionary from the checkpoint
model.load_state_dict(checkpoint['model'])

print(model) # The model has been quantized to int8

NET_32k_ECG(
  (quant): Quantize(scale=tensor([0.0376]), zero_point=tensor([46]), dtype=torch.quint8)
  (dequant): DeQuantize()
  (conv1_depthwise): QuantizedConv1d(1, 1, kernel_size=(7,), stride=(1,), scale=0.034143365919589996, zero_point=70, padding=(4,), dilation=(2,))
  (conv1_pointwise): QuantizedConv1d(1, 40, kernel_size=(1,), stride=(1,), scale=0.03150835260748863, zero_point=70)
  (bn1): QuantizedBatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation1): QuantizedHardswish()
  (conv2_depthwise): QuantizedConv1d(40, 40, kernel_size=(5,), stride=(1,), scale=0.07373173534870148, zero_point=66, padding=(1,), groups=40)
  (conv2_pointwise): QuantizedConv1d(40, 40, kernel_size=(1,), stride=(1,), scale=0.2259116917848587, zero_point=61)
  (activation2): QuantizedHardswish()
  (conv3_depthwise): QuantizedConv1d(40, 40, kernel_size=(3,), stride=(1,), scale=0.0875716507434845, zero_point=49, padding=(1,), groups=40)
  (conv3_pointwise): QuantizedCon

In [8]:
# Test the model on the test dataset and store the accuracy
model.eval()
test_accuracy = test_model(model, test_loader)

# Extract the int8 weights from all quantized layers in the model and save them to a list
quantized_weights_list = print_quantized_weights(model)

print(f"The test accuracy after quantization is: {test_accuracy:.2f}%")

Extracting weights from layer: conv1_depthwise
Weight Shape: (7,), Int8 Weights:
[   0    7   -4   46   27 -128   42]

Extracting weights from layer: conv1_pointwise
Weight Shape: (40,), Int8 Weights:
[ 127 -127  127  127  127 -128 -127  127 -128  127  127 -127  127 -128
 -127  127 -128  127  127  127  127 -127 -128  127 -128 -128  127  127
  127 -128  127  127 -127 -128 -127  127  127  127 -128 -128]

Extracting weights from layer: conv2_depthwise
Weight Shape: (200,), Int8 Weights:
[  64  -95   96   56 -128 -120   17  127   90   16 -128   99   85   34
  -14 -110  112   87 -128 -126 -128  -38   41   42    5   84  -69 -127
   80   54   16   -3   39  126 -128 -128  112  -82   46   -5   85   34
   15  -55 -127 -128   56   80   46   95  107  127   12  -51  -90  -62
 -106   91   -6 -128  127  -91  -31  -17   -8   69   31  127  -73  109
  -73    2   28   62  127  -46  -65 -127   63   -1  -44  127  -67  -91
 -104   77 -128   -2 -101   65    9  106   74 -128  -92 -127   45   -1
  127  127    